In [1]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects

from keras.layers import Input, Conv2D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.optimizers import SGD
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

from itertools import permutations

import seaborn as sns

from keras.datasets import mnist
from sklearn.manifold import TSNE

from sklearn.svm import SVC


In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
with open('Data/sub_dataset_list.pkl', 'rb') as f:
    smellKey_list = pickle.load(f)
smellKey_list

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
95    14.0
96    14.0
97    14.0
98    14.0
99    14.0
Name: smellKey_encoded, Length: 100, dtype: float64

In [18]:
series_data = pd.Series(np.arange(0, 2500, dtype=np.float64))
for i in range(len(series_data)):
  if i < 500:
    series_data[i] = 0
  elif i < 1000:
    series_data[i] = 1
  elif i < 1500:
    series_data[i] = 2
  elif i < 2000:
    series_data[i] = 3
  else:
    series_data[i] = 4


0.0

In [19]:
save_path = 'Data/2500_Smell.pt'
loaded_ast_embeddings = torch.load(save_path)
loaded_ast_embeddings = loaded_ast_embeddings.numpy()
loaded_ast_embeddings.shape

(2500, 768)

In [20]:
data = pd.DataFrame({
    'fn_smell_embeddings.pt': loaded_ast_embeddings.tolist(),
    'smellKey_encoded': series_data
})

X = np.array(data['fn_smell_embeddings.pt'].tolist())
y = np.array(data['smellKey_encoded'])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X.shape

(2500, 768)

In [ ]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [24]:
x_test.shape

array([[-0.21949668,  0.52009624,  0.80027395, ..., -0.39148009,
        -0.59952378,  0.4945178 ],
       [ 0.06018562,  0.44275349,  1.07437146, ..., -0.35398841,
        -0.79761583,  0.55565304],
       [ 0.08509637,  0.29572806,  0.819547  , ..., -0.53149134,
        -0.31133565,  0.34643734],
       ...,
       [ 0.07617371, -0.2423819 ,  1.13228214, ..., -0.39075366,
        -0.55751336,  0.64451575],
       [-0.15223742,  0.35884848, -0.09361228, ..., -0.59589565,
        -0.35234824,  0.83614349],
       [-0.04150327,  0.43444666,  0.55787909, ..., -0.32020199,
        -0.41934377,  0.42905429]])

In [ ]:
# Define our own plot function
def scatter(x, labels, subtitle=None):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[labels.astype(int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[labels == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)
        
    if subtitle != None:
        plt.suptitle(subtitle)
        
    plt.savefig(subtitle)

In [25]:
x_train_flat = x_train
x_test_flat = x_test
print(x_train.shape)
print(x_train_flat.shape)

(2000, 768)
(2000, 768)


In [ ]:
tsne = TSNE()
train_tsne_embeds = tsne.fit_transform(x_train[:512])
scatter(train_tsne_embeds, y_train[:512], "Samples from Training Data")

eval_tsne_embeds = tsne.fit_transform(x_test[:512])
scatter(eval_tsne_embeds, y_test[:512], "Samples from Validation Data")

In [19]:
# Classifier_input = Input((768,))
# Classifier_output = Dense(10, activation='softmax')(Classifier_input)
# Classifier_model = Model(Classifier_input, Classifier_output)

In [27]:
Classifier_model = Sequential()
Classifier_model.add(Dense(350 , input_shape=(768,), activation='relu'))
Classifier_model.add(Dense(50,activation='relu'))
Classifier_model.add(Dense(5, activation='softmax'))

In [28]:
from sklearn.preprocessing import LabelBinarizer

In [29]:
le = LabelBinarizer()

In [30]:
y_train_onehot = le.fit_transform(y_train)
y_test_onehot = le.transform(y_test)

In [31]:
Classifier_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
x_train_flat.shape

(2000, 768)

In [33]:
Classifier_model.fit(x_train_flat,y_train_onehot, validation_data=(x_test_flat,y_test_onehot),epochs=50)

Epoch 1/50
63/63 [==============================] - 1s 7ms/step - loss: 1.0468 - accuracy: 0.5890 - val_loss: 0.8243 - val_accuracy: 0.6840
Epoch 2/50
63/63 [==============================] - 0s 4ms/step - loss: 0.7341 - accuracy: 0.7150 - val_loss: 0.6538 - val_accuracy: 0.7460
Epoch 3/50
63/63 [==============================] - 0s 4ms/step - loss: 0.5947 - accuracy: 0.7790 - val_loss: 0.5489 - val_accuracy: 0.7920
Epoch 4/50
63/63 [==============================] - 0s 4ms/step - loss: 0.5083 - accuracy: 0.8225 - val_loss: 0.5188 - val_accuracy: 0.8180
Epoch 5/50
63/63 [==============================] - 0s 4ms/step - loss: 0.4762 - accuracy: 0.8300 - val_loss: 0.4974 - val_accuracy: 0.8280
Epoch 6/50
63/63 [==============================] - 0s 4ms/step - loss: 0.4237 - accuracy: 0.8505 - val_loss: 0.5430 - val_accuracy: 0.7780
Epoch 7/50
63/63 [==============================] - 0s 4ms/step - loss: 0.3794 - accuracy: 0.8585 - val_loss: 0.6143 - val_accuracy: 0.7680
Epoch 8/50
63/63 [==

In [34]:
def generate_triplet(x,y,testsize=0.3,ap_pairs=10,an_pairs=10):
    data_xy = tuple([x,y])

    trainsize = 1-testsize

    triplet_train_pairs = []
    triplet_test_pairs = []
    for data_class in sorted(set(data_xy[1])):

        same_class_idx = np.where((data_xy[1] == data_class))[0]
        diff_class_idx = np.where(data_xy[1] != data_class)[0]
        A_P_pairs = random.sample(list(permutations(same_class_idx,2)),k=ap_pairs) #Generating Anchor-Positive pairs
        Neg_idx = random.sample(list(diff_class_idx),k=an_pairs)
        

        #train
        A_P_len = len(A_P_pairs)
        Neg_len = len(Neg_idx)
        for ap in A_P_pairs[:int(A_P_len*trainsize)]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_train_pairs.append([Anchor,Positive,Negative])               
        #test
        for ap in A_P_pairs[int(A_P_len*trainsize):]:
            Anchor = data_xy[0][ap[0]]
            Positive = data_xy[0][ap[1]]
            for n in Neg_idx:
                Negative = data_xy[0][n]
                triplet_test_pairs.append([Anchor,Positive,Negative])    
                
    return np.array(triplet_train_pairs), np.array(triplet_test_pairs)

In [46]:
X_train, X_test = generate_triplet(x_train_flat,y_train, ap_pairs=150, an_pairs=150,testsize=0.2)
X_train.shape

(90000, 3, 768)

## Triplet NN

In [47]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """
    print('y_pred.shape = ',y_pred)
    
    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12
    
    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]

    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)

    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
 
    return loss

In [48]:
def create_base_network():
    """
    Base network to be shared.
    """
    model = Sequential()
    model.add(Flatten(input_shape=(768,1)))
    # model.add(Conv2D(128,(7,7),padding='same',input_shape=(in_dims[0],in_dims[1],in_dims[2],),activation='relu',name='conv1'))
    # model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool1'))
    # model.add(Conv2D(256,(5,5),padding='same',activation='relu',name='conv2'))
    # model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool2'))
    # model.add(Flatten(name='flatten'))
    model.add(Dense(768,name='embeddings'))
    # model.add(Dense(600))
    
    return model

In [49]:
adam_optim = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

In [50]:
anchor_input = Input((768,1), name='anchor_input')
positive_input = Input((768,1 ), name='positive_input')
negative_input = Input((768,1 ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network()


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')

model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)
model.compile(loss=triplet_loss, optimizer=adam_optim)

In [51]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor_input (InputLayer)      [(None, 768, 1)]     0           []                               
                                                                                                  
 positive_input (InputLayer)    [(None, 768, 1)]     0           []                               
                                                                                                  
 negative_input (InputLayer)    [(None, 768, 1)]     0           []                               
                                                                                                  
 sequential_3 (Sequential)      (None, 768)          590592      ['anchor_input[0][0]',           
                                                                  'positive_input[0][0]',   

In [52]:
X_train.shape

(90000, 3, 768)

In [53]:
Anchor = X_train[:,0,:].reshape(-1,768,1)
Positive = X_train[:,1,:].reshape(-1,768,1)
Negative = X_train[:,2,:].reshape(-1,768,1)
Anchor_test = X_test[:,0,:].reshape(-1,768,1)
Positive_test = X_test[:,1,:].reshape(-1,768,1)
Negative_test = X_test[:,2,:].reshape(-1,768,1)

Y_dummy = np.empty(( Anchor.shape[0],300))
Y_dummy2 = np.empty((Anchor_test.shape[0],1))

model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), batch_size=512, epochs=500)

Epoch 1/500
y_pred.shape =  Tensor("model_1/merged_layer/concat:0", shape=(None, 2304), dtype=float32)
y_pred.shape =  Tensor("model_1/merged_layer/concat:0", shape=(None, 2304), dtype=float32)
176/176 [==============================] - 9s 44ms/step - loss: 0.4374 - val_loss: 2.8371
Epoch 2/500
127/176 [====================>.........] - ETA: 1s - loss: 0.0150

KeyboardInterrupt: 

In [42]:
predict_ready_data = loaded_ast_embeddings.reshape(-1,768,1)
predict_ready_data.shape

(100, 768, 1)

In [43]:
predictedModel = model.predict([predict_ready_data, predict_ready_data, predict_ready_data])

4/4 [==============================] - 0s 3ms/step


In [44]:
predictedModel.shape

(100, 2304)

In [45]:
outputAnchor = predictedModel[:, :768]


In [ ]:
X[0]

In [ ]:
outputAnchor[0]

In [ ]:
outputAnchor.shape

In [46]:
x_train, x_test, y_train, y_test = train_test_split(outputAnchor, y, test_size=0.2, random_state=42)

In [47]:
Classifier_model.fit(x_train_flat,y_train_onehot, validation_data=(x_test_flat,y_test_onehot),epochs=50)

Epoch 1/50
3/3 [==============================] - 0s 31ms/step - loss: 0.0345 - accuracy: 0.9750 - val_loss: 1.6808 - val_accuracy: 0.5000
Epoch 2/50
3/3 [==============================] - 0s 16ms/step - loss: 0.0356 - accuracy: 0.9875 - val_loss: 1.6705 - val_accuracy: 0.5000
Epoch 3/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0353 - accuracy: 0.9875 - val_loss: 1.6911 - val_accuracy: 0.5000
Epoch 4/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0327 - accuracy: 0.9875 - val_loss: 1.7141 - val_accuracy: 0.4500
Epoch 5/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0349 - accuracy: 0.9875 - val_loss: 1.7167 - val_accuracy: 0.4500
Epoch 6/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0280 - accuracy: 0.9875 - val_loss: 1.7543 - val_accuracy: 0.4500
Epoch 7/50
3/3 [==============================] - 0s 15ms/step - loss: 0.0318 - accuracy: 0.9875 - val_loss: 1.7393 - val_accuracy: 0.4500
Epoch 8/50
3/3 [===========